In [2]:
 pip install requests


The following command must be run outside of the IPython shell:

    $ pip install requests

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [3]:
import requests
import pandas as pd

In [ ]:
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page =requests.get(url)
page.content

In [12]:
from lxml import html
doc =html.fromstring(page.content)
doc

<Element html at 0x7f61ce724bd8>

In [13]:
tr_elements = doc.xpath('//tr')

In [14]:
tr_elements = doc.xpath('//tr')
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


In [15]:
for j in range(1,len(tr_elements)):
    
    T=tr_elements[j]
    

    if len(T)!=3:
        break
    i=0
    

    for t in T.iterchildren():
        data=t.text_content() 
        
        if i>0:
        
            try:
                data=int(data)
            except:
                pass

        col[i][1].append(data)
    
        i+=1

In [16]:
[len(C) for (title,C) in col]

[181, 181, 181]

In [17]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [18]:
df.head()


,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [19]:
df.columns = ['Borough', 'Neighbourhood','Postcode']

cols = df.columns.tolist()
cols

cols = cols[-1:] + cols[:-1]

df = df[cols]

df.head()

,Postcode,Borough,Neighbourhood
0,Not assigned\n,M1A\n,Not assigned\n
1,Not assigned\n,M2A\n,Not assigned\n
2,Parkwoods\n,M3A\n,North York\n
3,Victoria Village\n,M4A\n,North York\n
4,"Regent Park, Harbourfront\n",M5A\n,Downtown Toronto\n


In [20]:
df = df.replace('\n',' ', regex=True)
df.head()

,Postcode,Borough,Neighbourhood
0,Not assigned,M1A,Not assigned
1,Not assigned,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,"Regent Park, Harbourfront",M5A,Downtown Toronto


In [21]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)
df = df.reset_index(drop=True)
df.head()

,Postcode,Borough,Neighbourhood
0,Not assigned,M1A,Not assigned
1,Not assigned,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,"Regent Park, Harbourfront",M5A,Downtown Toronto


In [22]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,,,Canadian postal codes
1,Agincourt,M1S,Scarborough
2,"Alderwood, Long Branch",M8W,Etobicoke
3,"Bathurst Manor, Wilson Heights, Downsview North",M3H,North York
4,Bayview Village,M2K,North York
5,"Bedford Park, Lawrence Manor East",M5M,North York
6,Berczy Park,M5E,Downtown Toronto
7,"Birch Cliff, Cliffside West",M1N,Scarborough
8,"Brockton, Parkdale Village, Exhibition Place",M6K,West Toronto
9,"Business reply mail Processing Centre, South C...",M7Y,East Toronto


In [24]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

In [25]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [28]:
df.shape

(181, 3)

In [29]:
df.to_csv(r'df_can.csv')